# Computational intelligence Project_2

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import KMeans
import tensorflow as tf

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Rescale the images from [0,255] to the [0.0,1.0] range.
# x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0
x_train, x_test = x_train.astype("float32") / 255.0, x_test.astype("float32") / 255.0

x_train, y_train = x_train[:6000], y_train[:6000]

print("x_train shape: ", x_train.shape)
print("y_train shape: ", y_train.shape)
print("Number of original training examples:", len(x_train))
print("Number of original test examples:", len(x_test))

x_train shape:  (6000, 28, 28)
y_train shape:  (6000,)
Number of original training examples: 6000
Number of original test examples: 10000


## K-Means:

In [3]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(x_train.reshape(6000, 28*28))

### K-Means rand index

In [4]:
metrics.rand_score(y_train, kmeans.labels_)

0.8760697338445297

In [5]:
metrics.rand_score(y_test, kmeans.predict(x_test.reshape(10000, 28*28)))

0.8732373237323733

### K-Means purity

In [6]:
def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix)

purity_score(y_train, kmeans.labels_)

0.5506666666666666

In [7]:
purity_score(y_test, kmeans.predict(x_test.reshape(10000, 28*28)))

0.5507

## MLP

In [26]:
# last layer sigmoid: rand index ~= 10, purity ~= 14
# last layer none: rand index ~= 29, purity ~= 30
# last layer softmax: rand index ~= 24, purity ~= 25

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(256, activation='sigmoid'),
  tf.keras.layers.Dense(64, activation='sigmoid'),
  tf.keras.layers.Dense(10)
])

In [27]:
model.compile(optimizer='sgd',
              # loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

### callbacks

In [ ]:
predictions = np.zeros((6000, 10), dtype=np.float32)
# predictions = np.random.rand(6000, 10) / .2

temp_y = np.random.randint(0, 10, 6000)
epochs = 10

class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        global temp_y, predictions
        if epoch % epochs == 0 and epoch != 0:
            # print(epoch)
            np.argmax(predictions, axis=1, out=temp_y)
            # predictions = np.random.rand(6000, 10) / .2
            predictions = np.zeros((6000, 10), dtype=np.float32)
        else:
            prediction = model.predict(x_train)
            predictions = np.add(predictions, prediction)

### predictions

In [28]:
# alpha = .2
alpha = 1

avg_x = np.zeros((10, 28, 28), dtype=np.float32)
avg_y = np.array([i for i in range(10)])

def learn_turn(turn_num):
    global predictions, avg_x
    model.fit(
        x_train,
        temp_y,
        batch_size=64,
        epochs=50,
        verbose=0,
        callbacks=[MyCallback()],
    )
    avg_x = np.zeros((10, 28, 28), dtype=np.float32)
    for idx, val in enumerate(temp_y):
        avg_x[val] = np.add(avg_x[val], x_train[idx])
    model.fit(
        avg_x,
        avg_y,
        batch_size=1,
        epochs=20,
        verbose=0,
        # callbacks=[MyCallback()],
    )


In [29]:
for i in range(5):
    print("turn {}".format(i+1))
    learn_turn(i)

model.fit(
    x_train,
    temp_y,
    batch_size=64,
    epochs=500,
    verbose=2,
    callbacks=[MyCallback()],
)

turn 1
turn 2
turn 3
turn 4
turn 5
turn 6
turn 7
turn 8
turn 9
turn 10
turn 11
turn 12
turn 13
turn 14
turn 15
turn 16
turn 17
turn 18
turn 19
turn 20
turn 21
turn 22
turn 23
turn 24
turn 25
turn 26
turn 27
turn 28
turn 29
turn 30


### MLP rand index and purity

In [30]:
model.evaluate(x_train,  temp_y)

188/188 [==============================] - 0s 739us/step - loss: 2.3052 - accuracy: 0.1037


[2.3051586151123047, 0.10366666316986084]

In [31]:
metrics.rand_score(y_train, temp_y)

0.8197060065566484

In [32]:
result = model.predict(x_test)
result = np.argmax(result, axis=1)
metrics.rand_score(y_test, result)

0.7047044904490449

In [33]:
purity_score(y_test, result)

0.2689

In [34]:
purity_score(y_train, temp_y)

0.12016666666666667